In [18]:
from sklearn import tree
import pandas as pd
import os

In [19]:
activities_csv = pd.read_csv('ac_activities_torben_clean_v2.csv')
activities_csv.head()

,id,type,start_date_local,x_start_h,name,name_short,x_min,x_mi,x_min/mi,x_mph,...,athlete_count,average_temp,distance,elapsed_time,start_latitude,start_longitude,timezone,x_date,x_elev_%,x_url
0,868156484,Run,15.01.2017 06:17:41,6.3,Morning Run,Morning,248.2,26.510,9.36,6.41,...,29,NaN,42663.3,14892,33.176906,-117.335854,(GMT-08:00) America/Los_Angeles,2017-01-15,0.51,https://www.strava.com/activities/868156484
1,868156415,Run,17.01.2017 18:50:18,18.8,Evening Run,Evening,16.1,2.023,7.95,7.55,...,30,NaN,3256.2,1300,32.720301,-117.150205,(GMT-08:00) America/Los_Angeles,2017-01-17,NaN,https://www.strava.com/activities/868156415
2,868156459,Run,19.01.2017 06:35:12,6.6,Morning Run,Morning,87.0,10.091,8.62,6.96,...,1,NaN,16240.6,5271,32.853794,-116.960111,(GMT-08:00) America/Los_Angeles,2017-01-19,0.85,https://www.strava.com/activities/868156459
3,868156435,Run,21.01.2017 07:17:12,7.3,Morning Run,Morning,88.8,11.518,7.71,7.78,...,2,NaN,18535.6,5804,32.758368,-117.242017,(GMT-08:00) America/Los_Angeles,2017-01-21,0.16,https://www.strava.com/activities/868156435
4,868156458,Run,22.01.2017 16:50:51,16.8,Afternoon Run,Afternoon,89.6,10.066,8.90,6.74,...,1,NaN,16199.8,5658,32.853570,-116.960471,(GMT-08:00) America/Los_Angeles,2017-01-22,0.48,https://www.strava.com/activities/868156458


In [20]:
activities = activities_csv[["x_start_h", "name_short", "x_min", "x_mi", "x_min/mi", "total_elevation_gain", "average_cadence", "elev_low", "elev_high", "athlete_count", "distance", "elapsed_time"]].copy()
activities["pace"] = activities["x_min/mi"]
activities.head()

,x_start_h,name_short,x_min,x_mi,x_min/mi,total_elevation_gain,average_cadence,elev_low,elev_high,athlete_count,distance,elapsed_time,pace
0,6.3,Morning,248.2,26.510,9.36,217.6,80.8,-7.1,97.1,29,42663.3,14892,9.36
1,18.8,Evening,16.1,2.023,7.95,0.0,84.5,40.7,44.6,30,3256.2,1300,7.95
2,6.6,Morning,87.0,10.091,8.62,137.6,84.0,106.0,201.5,1,16240.6,5271,8.62
3,7.3,Morning,88.8,11.518,7.71,29.5,84.9,1.3,20.1,2,18535.6,5804,7.71
4,16.8,Afternoon,89.6,10.066,8.90,77.0,83.0,96.0,137.9,1,16199.8,5658,8.90


In [21]:
activities = activities.dropna(how="any")
#activities["start_date_local"] = activities["start_date_local"].apply(pd.to_datetime,errors="coerce")
activities.dtypes

x_start_h               float64
name_short               object
x_min                   float64
x_mi                    float64
x_min/mi                float64
total_elevation_gain    float64
average_cadence         float64
elev_low                float64
elev_high               float64
athlete_count             int64
distance                float64
elapsed_time              int64
pace                    float64
dtype: object

In [22]:
activities.head()

,x_start_h,name_short,x_min,x_mi,x_min/mi,total_elevation_gain,average_cadence,elev_low,elev_high,athlete_count,distance,elapsed_time,pace
0,6.3,Morning,248.2,26.510,9.36,217.6,80.8,-7.1,97.1,29,42663.3,14892,9.36
1,18.8,Evening,16.1,2.023,7.95,0.0,84.5,40.7,44.6,30,3256.2,1300,7.95
2,6.6,Morning,87.0,10.091,8.62,137.6,84.0,106.0,201.5,1,16240.6,5271,8.62
3,7.3,Morning,88.8,11.518,7.71,29.5,84.9,1.3,20.1,2,18535.6,5804,7.71
4,16.8,Afternoon,89.6,10.066,8.90,77.0,83.0,96.0,137.9,1,16199.8,5658,8.90


In [23]:
target = activities["name_short"]
target_names = ["Morning", "Afternoon","Evening"]
target.head()

0      Morning
1      Evening
2      Morning
3      Morning
4    Afternoon
Name: name_short, dtype: object

In [24]:
data = activities.drop(["name_short", "x_min/mi", "distance", "elapsed_time"], axis=1)
feature_names = data.columns
data.head()

,x_start_h,x_min,x_mi,total_elevation_gain,average_cadence,elev_low,elev_high,athlete_count,pace
0,6.3,248.2,26.510,217.6,80.8,-7.1,97.1,29,9.36
1,18.8,16.1,2.023,0.0,84.5,40.7,44.6,30,7.95
2,6.6,87.0,10.091,137.6,84.0,106.0,201.5,1,8.62
3,7.3,88.8,11.518,29.5,84.9,1.3,20.1,2,7.71
4,16.8,89.6,10.066,77.0,83.0,96.0,137.9,1,8.90


In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [26]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.9908256880733946

In [27]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.981651376146789

In [28]:
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

[(0.6512897466301141, 'x_start_h'),
 (0.07400640054949353, 'x_mi'),
 (0.07294848299752424, 'x_min'),
 (0.051190380584399155, 'average_cadence'),
 (0.043226402511375966, 'elev_low'),
 (0.042828106277288724, 'elev_high'),
 (0.03841581106832941, 'total_elevation_gain'),
 (0.022954509799516548, 'pace'),
 (0.003140159581958344, 'athlete_count')]

In [29]:
data = activities.drop(["name_short", "x_min/mi", "distance", "elapsed_time", "x_start_h"], axis=1)
feature_names = data.columns
data.head()

,x_min,x_mi,total_elevation_gain,average_cadence,elev_low,elev_high,athlete_count,pace
0,248.2,26.510,217.6,80.8,-7.1,97.1,29,9.36
1,16.1,2.023,0.0,84.5,40.7,44.6,30,7.95
2,87.0,10.091,137.6,84.0,106.0,201.5,1,8.62
3,88.8,11.518,29.5,84.9,1.3,20.1,2,7.71
4,89.6,10.066,77.0,83.0,96.0,137.9,1,8.90


In [30]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [31]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.6513761467889908

In [32]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.6146788990825688

In [33]:
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

[(0.17716325531618315, 'x_min'),
 (0.16347177098908555, 'x_mi'),
 (0.13830246897604165, 'elev_high'),
 (0.13787352711623674, 'average_cadence'),
 (0.13589721271505234, 'elev_low'),
 (0.116534758600501, 'total_elevation_gain'),
 (0.10629316971003244, 'pace'),
 (0.024463836576867085, 'athlete_count')]